In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder
import os

# Set style for plots
plt.style.use('ggplot')
sns.set(font_scale=1.2)

# Create output directory for plots
os.makedirs('plots', exist_ok=True)

# Load the dataset
df = pd.read_csv('vgchartz_cleaned.csv')

# Basic info and statistical summary
print("Dataset Shape:", df.shape)
print("\nData Types:")
print(df.dtypes)

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Convert release_year to numeric if needed
if df['release_year'].dtype == 'object':
    df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce')

# Identify variable types
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print("\nNumeric columns:", numeric_cols)
print("\nCategorical columns:", categorical_cols)

# Function to identify data type
def identify_variable_type(col):
    if df[col].dtype == 'object':
        unique_values = df[col].nunique()
        if unique_values == 2:
            return "Binary"
        elif unique_values <= 10:
            return "Nominal" if not all(df[col].dropna().astype(str).str.isnumeric()) else "Ordinal"
        else:
            return "Nominal"
    else:  # numeric
        unique_values = df[col].nunique()
        if unique_values == 2:
            return "Binary"
        elif unique_values <= 10:
            return "Ordinal"
        else:
            return "Numeric (Continuous)"

# Identify variable types
print("\nVariable Types:")
for col in df.columns:
    print(f"{col}: {identify_variable_type(col)}")

# Calculate statistics for numeric variables
def calculate_statistics(df, column):
    if column in df.columns:
        data = df[column].dropna()
        
        if pd.api.types.is_numeric_dtype(data):
            # Calculate statistics
            mean = data.mean()
            median = data.median()
            mode = data.mode()[0]
            midrange = (data.max() + data.min()) / 2
            q1 = data.quantile(0.25)
            q3 = data.quantile(0.75)
            min_val = data.min()
            max_val = data.max()
            
            print(f"\nStatistics for {column}:")
            print(f"Mean: {mean}")
            print(f"Median: {median}")
            print(f"Mode: {mode}")
            print(f"Midrange: {midrange}")
            print(f"Five Number Summary:")
            print(f"  Minimum: {min_val}")
            print(f"  Q1: {q1}")
            print(f"  Median: {median}")
            print(f"  Q3: {q3}")
            print(f"  Maximum: {max_val}")
            
            return {
                'mean': mean,
                'median': median,
                'mode': mode,
                'midrange': midrange,
                'min': min_val,
                'q1': q1,
                'q3': q3,
                'max': max_val
            }
        else:
            print(f"\n{column} is not numeric, skipping statistics.")
            return None
    else:
        print(f"\n{column} not found in dataset.")
        return None

# Create the 10 required plots with gaming dataset adaptations
def create_plots(df):
    # 1. Histogram for total_sales
    plt.figure(figsize=(10, 6))
    sns.histplot(df['total_sales'], kde=True)
    plt.title('Histogram of Total Game Sales')
    plt.xlabel('Total Sales (millions)')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.savefig('plots/1_histogram_total_sales.png')
    plt.close()
    print("1. Created histogram for total sales")
    
    # 2. Box Plot for sales across regions
    sales_cols = ['na_sales', 'jp_sales', 'pal_sales', 'other_sales']
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df[sales_cols])
    plt.title('Box Plot of Sales by Region')
    plt.ylabel('Sales (millions)')
    plt.tight_layout()
    plt.savefig('plots/2_boxplot_regional_sales.png')
    plt.close()
    print("2. Created box plot for regional sales")
    
    # 3. QQ Plot for critic scores
    plt.figure(figsize=(8, 8))
    stats.probplot(df['critic_score'].dropna(), dist="norm", plot=plt)
    plt.title('Q-Q Plot of Critic Scores')
    plt.tight_layout()
    plt.savefig('plots/3_qqplot_critic_score.png')
    plt.close()
    print("3. Created QQ Plot for critic scores")
    
    # 4. Correlation Heatmap for numeric columns
    numeric_game_cols = ['critic_score', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales', 'release_year']
    plt.figure(figsize=(12, 10))
    corr = df[numeric_game_cols].corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    sns.heatmap(corr, mask=mask, annot=True, cmap='coolwarm', linewidths=0.5)
    plt.title('Correlation Heatmap of Sales and Scores')
    plt.tight_layout()
    plt.savefig('plots/4_correlation_heatmap.png')
    plt.close()
    print("4. Created correlation heatmap")
    
    # 5. Scatter Plot between critic_score and total_sales
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='critic_score', y='total_sales', data=df)
    plt.title('Scatter Plot: Critic Score vs Total Sales')
    plt.xlabel('Critic Score')
    plt.ylabel('Total Sales (millions)')
    plt.tight_layout()
    plt.savefig('plots/5_scatterplot_score_sales.png')
    plt.close()
    print("5. Created scatter plot between critic score and total sales")
    
    # 6. Bar Chart for game genres
    plt.figure(figsize=(14, 8))
    genre_counts = df['genre'].value_counts().sort_values(ascending=False)
    sns.barplot(x=genre_counts.index, y=genre_counts.values)
    plt.title('Number of Games by Genre')
    plt.xlabel('Genre')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('plots/6_barchart_genres.png')
    plt.close()
    print("6. Created bar chart for game genres")
    
    # 7. Pie Chart for console distribution
    plt.figure(figsize=(10, 10))
    df['console'].value_counts().plot.pie(autopct='%1.1f%%')
    plt.title('Distribution of Games by Console')
    plt.ylabel('')
    plt.tight_layout()
    plt.savefig('plots/7_piechart_consoles.png')
    plt.close()
    print("7. Created pie chart for console distribution")
    
    # 8. Violin Plot for total sales by genre
    plt.figure(figsize=(16, 8))
    # Limit to top 10 genres if there are many
    top_genres = df['genre'].value_counts().nlargest(10).index
    genre_data = df[df['genre'].isin(top_genres)]
    sns.violinplot(x='genre', y='total_sales', data=genre_data)
    plt.title('Total Sales Distribution by Genre')
    plt.xlabel('Genre')
    plt.ylabel('Total Sales (millions)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('plots/8_violinplot_sales_by_genre.png')
    plt.close()
    print("8. Created violin plot for total sales by genre")
    
    # 9. Quantile Plot (Empirical CDF) for total sales
    plt.figure(figsize=(10, 6))
    sorted_data = np.sort(df['total_sales'].dropna())
    y = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
    plt.plot(sorted_data, y, marker='.', linestyle='none')
    plt.title('Quantile Plot (Empirical CDF) of Total Sales')
    plt.xlabel('Total Sales (millions)')
    plt.ylabel('Cumulative Probability')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('plots/9_quantile_plot_total_sales.png')
    plt.close()
    print("9. Created quantile plot for total sales")
    
    # 10. Pair Plot for sales across regions
    plt.figure(figsize=(16, 12))
    sns.pairplot(df[['na_sales', 'jp_sales', 'pal_sales', 'other_sales', 'critic_score']], height=2.5)
    plt.suptitle('Pair Plot of Regional Sales and Critic Score', y=1.02)
    plt.tight_layout()
    plt.savefig('plots/10_pairplot_sales.png')
    plt.close()
    print("10. Created pair plot for regional sales and critic score")

    # 11. Line plot of game releases by year
    plt.figure(figsize=(14, 8))
    year_counts = df.groupby('release_year').size()
    year_counts.plot(kind='line', marker='o')
    plt.title('Number of Games Released by Year')
    plt.xlabel('Release Year')
    plt.ylabel('Number of Games')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('plots/11_lineplot_releases_by_year.png')
    plt.close()
    print("11. Created line plot of game releases by year")
    
    # 12. Heatmap of genre popularity over time
    plt.figure(figsize=(16, 10))
    genre_year = df.groupby(['release_year', 'genre']).size().unstack().fillna(0)
    # Limit to top 10 genres if needed
    if len(genre_year.columns) > 10:
        top_genres = df['genre'].value_counts().nlargest(10).index
        genre_year = genre_year[genre_year.columns.intersection(top_genres)]
    sns.heatmap(genre_year.T, cmap='YlGnBu', linewidths=0.5)
    plt.title('Genre Popularity Over Years')
    plt.xlabel('Release Year')
    plt.ylabel('Genre')
    plt.tight_layout()
    plt.savefig('plots/12_heatmap_genre_by_year.png')
    plt.close()
    print("12. Created heatmap of genre popularity over time")

# Calculate statistics for all numeric columns
numeric_columns = ['critic_score', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales', 'release_year']
for col in numeric_columns:
    calculate_statistics(df, col)

# Calculate top publishers by total sales
top_publishers = df.groupby('publisher')['total_sales'].sum().sort_values(ascending=False).head(10)
print("\nTop 10 Publishers by Total Sales:")
for publisher, sales in top_publishers.items():
    print(f"{publisher}: {sales:.2f} million")

# Calculate top selling games
top_games = df.sort_values('total_sales', ascending=False).head(10)[['title', 'console', 'total_sales']]
print("\nTop 10 Best-Selling Games:")
for i, (_, row) in enumerate(top_games.iterrows(), 1):
    print(f"{i}. {row['title']} ({row['console']}): {row['total_sales']:.2f} million")

# Generate the plots
create_plots(df)

print("\nAnalysis complete. All plots have been saved to the 'plots' directory.") 

Dataset Shape: (18874, 14)

Data Types:
title            object
console          object
genre            object
publisher        object
developer        object
critic_score    float64
total_sales     float64
na_sales        float64
jp_sales        float64
pal_sales       float64
other_sales     float64
release_date     object
last_update      object
release_year    float64
dtype: object

Missing Values:
title               0
console             0
genre               0
publisher           0
developer           4
critic_score        0
total_sales         0
na_sales         6252
jp_sales        12176
pal_sales        6067
other_sales      3762
release_date       90
last_update     14871
release_year       90
dtype: int64

Numeric columns: ['critic_score', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales', 'release_year']

Categorical columns: ['title', 'console', 'genre', 'publisher', 'developer', 'release_date', 'last_update']

Variable Types:
title: Nominal
console: Nomi

<Figure size 1600x1200 with 0 Axes>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
import os

# Create directory for processed data and visualizations
os.makedirs('processed_data', exist_ok=True)
os.makedirs('cleaning_plots', exist_ok=True)

print("Loading the dataset...")
# Load the dataset and create a copy for preprocessing
df = pd.read_csv('vgchartz_cleaned.csv')
df_processed = df.copy()

print(f"Original dataset shape: {df.shape}")

# ============= DATA CLEANING =============
print("\n===== DATA CLEANING =====")

# 1. Check for duplicate records
duplicates = df_processed.duplicated().sum()
print(f"Number of duplicate records: {duplicates}")
if duplicates > 0:
    df_processed = df_processed.drop_duplicates()
    print(f"After removing duplicates: {df_processed.shape}")

# 2. Check for missing values
print("\nMissing Values:")
missing_values = df_processed.isnull().sum()
print(missing_values)

# Only keep rows where essential columns are not null
essential_columns = ['title', 'console', 'genre', 'publisher', 'critic_score', 'total_sales']
df_processed = df_processed.dropna(subset=essential_columns)
print(f"After removing rows with missing essential data: {df_processed.shape}")

# Convert release_year to numeric if needed
if df_processed['release_year'].dtype == 'object':
    df_processed['release_year'] = pd.to_numeric(df_processed['release_year'], errors='coerce')

# 3. Check for inconsistent values in numeric columns
numeric_cols = ['critic_score', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 
                'other_sales', 'release_year']

print("\nChecking for outliers and inconsistent values in numeric columns...")
for col in numeric_cols:
    if col in df_processed.columns:
        # Skip columns with too many NaN values
        if df_processed[col].isna().sum() > len(df_processed) * 0.5:
            print(f"Skipping {col} due to high missing value rate")
            continue
            
        # Print range and check for unusual values
        min_val = df_processed[col].min()
        max_val = df_processed[col].max()
        print(f"{col}: Range [{min_val}, {max_val}]")
        
        # Identify potential outliers using IQR method
        Q1 = df_processed[col].quantile(0.25)
        Q3 = df_processed[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df_processed[(df_processed[col] < lower_bound) | (df_processed[col] > upper_bound)]
        
        if not outliers.empty:
            print(f"  Found {len(outliers)} potential outliers in {col}")
            
            # Visualize outliers
            plt.figure(figsize=(10, 6))
            sns.boxplot(x=df_processed[col])
            plt.title(f'Boxplot of {col} - Outlier Detection')
            plt.tight_layout()
            plt.savefig(f'cleaning_plots/outliers_{col}.png')
            plt.close()
            
            # Cap outliers to the boundary values (alternative to dropping)
            df_processed[col] = df_processed[col].clip(lower_bound, upper_bound)
            print(f"  Outliers in {col} capped to [{lower_bound:.2f}, {upper_bound:.2f}]")

# 4. Check for logical inconsistencies
print("\nChecking for logical inconsistencies...")

# Critic score should be between 0 and 10
if df_processed['critic_score'].max() > 10 or df_processed['critic_score'].min() < 0:
    print(f"Found critic_score values outside the expected range [0, 10]")
    df_processed['critic_score'] = df_processed['critic_score'].clip(0, 10)
    print("  Capped critic_score to [0, 10]")

# Sales figures should be non-negative
for col in ['total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales']:
    if col in df_processed.columns and df_processed[col].min() < 0:
        print(f"Found negative values in {col}")
        df_processed[col] = df_processed[col].clip(0, None)
        print(f"  Capped {col} to non-negative values")

# Release year should be reasonable (e.g., 1970-2023)
if 'release_year' in df_processed.columns:
    invalid_years = df_processed[(df_processed['release_year'] < 1970) | (df_processed['release_year'] > 2023)]
    if not invalid_years.empty:
        print(f"Found {len(invalid_years)} records with unusual release_year values")
        df_processed['release_year'] = df_processed['release_year'].clip(1970, 2023)
        print("  Capped release_year to [1970, 2023]")

# 5. Fill remaining missing numeric values
print("\nFilling remaining missing values...")
for col in numeric_cols:
    if col in df_processed.columns and df_processed[col].isna().any():
        missing_count = df_processed[col].isna().sum()
        df_processed[col] = df_processed[col].fillna(df_processed[col].median())
        print(f"  Filled {missing_count} missing values in {col} with median")

# ============= DATA TRANSFORMATION =============
print("\n===== DATA TRANSFORMATION =====")

# 1. Standardize numeric features
print("\nStandardizing numeric features...")
numeric_cols_to_scale = [col for col in numeric_cols if col in df_processed.columns]
scaler = StandardScaler()
df_processed[numeric_cols_to_scale] = scaler.fit_transform(df_processed[numeric_cols_to_scale])
print(f"Standardized {len(numeric_cols_to_scale)} numeric columns")

# 2. Encoding categorical variables
categorical_cols = ['console', 'genre', 'publisher', 'developer']
print("\nEncoding categorical variables...")

for col in categorical_cols:
    if col in df_processed.columns:
        print(f"Processing {col}...")
        # Check number of unique values
        n_unique = df_processed[col].nunique()
        print(f"  {n_unique} unique values")
        
        # For binary variables, use simple label encoding
        if n_unique == 2:
            le = LabelEncoder()
            df_processed[col] = le.fit_transform(df_processed[col])
            print(f"  Applied label encoding to {col}")
            mapping = dict(zip(le.classes_, le.transform(le.classes_)))
            print(f"  Mapping: {mapping}")
        
        # For variables with limited categories, use one-hot encoding
        elif n_unique <= 15:
            # Get dummies and drop the first to avoid multicollinearity
            dummies = pd.get_dummies(df_processed[col], prefix=col, drop_first=True)
            df_processed = pd.concat([df_processed, dummies], axis=1)
            df_processed.drop(col, axis=1, inplace=True)
            print(f"  Applied one-hot encoding to {col}, created {len(dummies.columns)} new features")
        
        # For high-cardinality variables, consider frequency encoding
        else:
            # Create a frequency map
            freq_map = df_processed[col].value_counts(normalize=True).to_dict()
            df_processed[f'{col}_freq'] = df_processed[col].map(freq_map)
            df_processed.drop(col, axis=1, inplace=True)
            print(f"  Applied frequency encoding to {col}")

# 3. Feature Engineering
print("\nCreating engineered features...")

# Calculate sales ratios between regions
if all(col in df_processed.columns for col in ['na_sales', 'total_sales']):
    df_processed['na_sales_ratio'] = df_processed['na_sales'] / df_processed['total_sales']
    print("Created na_sales_ratio (NA sales / total sales)")

if all(col in df_processed.columns for col in ['jp_sales', 'total_sales']):
    df_processed['jp_sales_ratio'] = df_processed['jp_sales'] / df_processed['total_sales']
    print("Created jp_sales_ratio (Japan sales / total sales)")

if all(col in df_processed.columns for col in ['pal_sales', 'total_sales']):
    df_processed['pal_sales_ratio'] = df_processed['pal_sales'] / df_processed['total_sales']
    print("Created pal_sales_ratio (PAL sales / total sales)")

# Create age of game feature
if 'release_year' in df_processed.columns:
    current_year = 2023
    df_processed['game_age'] = current_year - df_processed['release_year']
    print("Created game_age feature (years since release)")

# Create sales per year feature
if all(col in df_processed.columns for col in ['total_sales', 'game_age']):
    df_processed['sales_per_year'] = df_processed['total_sales'] / (df_processed['game_age'] + 1)  # +1 to avoid division by zero
    print("Created sales_per_year feature")

# Create critic score categorical variable
if 'critic_score' in df_processed.columns:
    bins = [0, 5, 7, 8, 9, 10]
    labels = ['Poor', 'Average', 'Good', 'Great', 'Excellent']
    df_processed['critic_score_category'] = pd.cut(df_processed['critic_score'], bins=bins, labels=labels)
    dummies = pd.get_dummies(df_processed['critic_score_category'], prefix='rating')
    df_processed = pd.concat([df_processed, dummies], axis=1)
    df_processed.drop('critic_score_category', axis=1, inplace=True)
    print("Created critic_score category and dummy variables")

# 4. Remove any remaining NaN values
df_processed = df_processed.dropna()
print(f"\nAfter removing remaining NaN values: {df_processed.shape}")

# 5. Save the processed data
processed_file = 'processed_data/vgchartz_processed.csv'
df_processed.to_csv(processed_file, index=False)
print(f"\nProcessed data saved to {processed_file}")
print(f"Final dataset shape: {df_processed.shape}")

# Summary of transformations
print("\nSummary of Transformations Applied:")
print("1. Removed duplicates")
print("2. Handled missing values in essential columns")
print("3. Capped outliers in numeric columns")
print("4. Fixed logical inconsistencies")
print("5. Standardized numeric features")
print("6. Encoded categorical variables")
print("7. Created engineered features")

# Print head of processed dataset
print("\nFirst few rows of processed dataset:")
print(df_processed.head(3))
print("\nColumn list of processed dataset:")
print(df_processed.columns.tolist()) 

Loading the dataset...
Original dataset shape: (18874, 14)

===== DATA CLEANING =====
Number of duplicate records: 0

Missing Values:
title               0
console             0
genre               0
publisher           0
developer           4
critic_score        0
total_sales         0
na_sales         6252
jp_sales        12176
pal_sales        6067
other_sales      3762
release_date       90
last_update     14871
release_year       90
dtype: int64
After removing rows with missing essential data: (18874, 14)

Checking for outliers and inconsistent values in numeric columns...
critic_score: Range [1.0, 10.0]
  Found 3956 potential outliers in critic_score
  Outliers in critic_score capped to [7.50, 7.50]
total_sales: Range [0.0, 20.32]
  Found 1905 potential outliers in total_sales
  Outliers in total_sales capped to [-0.45, 0.83]
na_sales: Range [0.0, 9.76]
  Found 1210 potential outliers in na_sales
  Outliers in na_sales capped to [-0.30, 0.63]
Skipping jp_sales due to high missing

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os

# Create directory for clustering results
os.makedirs('clustering_results', exist_ok=True)

# Load the preprocessed data
print("Loading preprocessed video game sales dataset...")
df = pd.read_csv('processed_data/vgchartz_processed.csv')
print(f"Dataset shape: {df.shape}")

# Ensure we only have numeric data for clustering
print("\nChecking for non-numeric columns...")
numeric_df = df.select_dtypes(include=['float64', 'int64'])
numeric_columns = numeric_df.columns.tolist()
print(f"Found {len(numeric_columns)} numeric columns out of {len(df.columns)} total columns")

# Also exclude any identifier columns if present
features = [col for col in numeric_columns if 'id' not in col.lower()]
print(f"Selected {len(features)} features for clustering")
print("Features:", features)

# Use a subset of features if there are too many
if len(features) > 15:
    print("\nToo many features, selecting most important ones...")
    # Prioritize key metrics for clustering
    key_features = [col for col in features if any(term in col.lower() for term in 
                   ['sales', 'score', 'year', 'age', 'ratio'])]
    if len(key_features) >= 5:  # Ensure we have a reasonable number of features
        features = key_features
        print(f"Using {len(features)} key features")
        print("Key features:", features)

# Verify there are no NaN values in the selected features
X_df = df[features].copy()
nan_counts = X_df.isna().sum()
if nan_counts.sum() > 0:
    print("\nWarning: Found NaN values in features. Filling with feature means.")
    X_df = X_df.fillna(X_df.mean())

# Check for string values that might have been encoded as objects
for col in X_df.columns:
    if X_df[col].dtype == 'object':
        print(f"Converting column {col} to numeric, errors will be set to NaN")
        X_df[col] = pd.to_numeric(X_df[col], errors='coerce')
        X_df[col] = X_df[col].fillna(X_df[col].mean())

X = X_df.values

# Determine optimal number of clusters using Elbow Method
inertia = []
silhouette_scores = []
max_clusters = 10

print("\nFinding optimal number of clusters using Elbow Method...")
for k in range(2, max_clusters + 1):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
    
    # Calculate silhouette score (only if k > 1)
    if k > 1:
        labels = kmeans.labels_
        silhouette_avg = silhouette_score(X, labels)
        silhouette_scores.append(silhouette_avg)
        print(f"For n_clusters = {k}, silhouette score is {silhouette_avg:.3f}")

# Plot Elbow Method results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(2, max_clusters + 1), inertia, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(2, max_clusters + 1), silhouette_scores, marker='o')
plt.title('Silhouette Score Method')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.grid(True)

plt.tight_layout()
plt.savefig('clustering_results/kmeans_elbow_method.png')
plt.close()

# Based on the elbow method and silhouette scores, choose the optimal number of clusters
# This is a simple heuristic - in practice, you would analyze the plots and choose accordingly
optimal_k = silhouette_scores.index(max(silhouette_scores)) + 2  # +2 because we started from k=2
print(f"\nOptimal number of clusters based on silhouette score: {optimal_k}")

# Perform K-means clustering with the optimal number of clusters
print(f"\nPerforming K-means clustering with {optimal_k} clusters...")
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df_cluster = df.copy()
df_cluster['Cluster'] = kmeans.fit_predict(X)

# Count instances in each cluster
cluster_counts = df_cluster['Cluster'].value_counts().sort_index()
print("\nCluster distribution:")
for cluster, count in cluster_counts.items():
    print(f"Cluster {cluster}: {count} instances ({count/len(df_cluster)*100:.2f}%)")

# Analyze clusters
print("\nAnalyzing clusters...")
cluster_analysis = df_cluster.groupby('Cluster')[features].mean()
print("\nCluster centers (mean values for each feature):")
print(cluster_analysis)

# Save cluster analysis to CSV
cluster_analysis.to_csv('clustering_results/kmeans_cluster_analysis.csv')

# Visualize clusters using PCA for dimensionality reduction
print("\nVisualizing clusters using PCA...")
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df_cluster['Cluster'], cmap='viridis', alpha=0.7)
plt.colorbar(scatter, label='Cluster')
plt.title(f'Video Game Clusters Visualization with PCA (K-means, k={optimal_k})')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.grid(True, alpha=0.3)
plt.savefig('clustering_results/kmeans_pca_visualization.png')
plt.close()

# Analyze relationship between clusters and total sales
print("\nAnalyzing relationship between clusters and total sales...")
if 'total_sales' in features:
    sales_by_cluster = df_cluster.groupby('Cluster')['total_sales'].mean().sort_values()
    print("\nAverage sales by cluster:")
    print(sales_by_cluster)

    plt.figure(figsize=(10, 6))
    sales_by_cluster.plot(kind='bar')
    plt.title('Average Total Sales by Cluster')
    plt.xlabel('Cluster')
    plt.ylabel('Average Sales')
    plt.axhline(y=df_cluster['total_sales'].mean(), color='red', linestyle='--', label='Overall Average')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('clustering_results/kmeans_sales_by_cluster.png')
    plt.close()

# Analyze relationship between clusters and critic scores if available
if 'critic_score' in features:
    scores_by_cluster = df_cluster.groupby('Cluster')['critic_score'].mean().sort_values()
    print("\nAverage critic scores by cluster:")
    print(scores_by_cluster)

    plt.figure(figsize=(10, 6))
    scores_by_cluster.plot(kind='bar')
    plt.title('Average Critic Score by Cluster')
    plt.xlabel('Cluster')
    plt.ylabel('Average Critic Score')
    plt.axhline(y=df_cluster['critic_score'].mean(), color='red', linestyle='--', label='Overall Average')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('clustering_results/kmeans_scores_by_cluster.png')
    plt.close()

# Analyze key features by cluster
# Select a subset of important features to visualize
if len(features) > 10:
    # Get the most variable features across clusters
    feature_variance = cluster_analysis.var().sort_values(ascending=False)
    key_features = feature_variance.head(10).index.tolist()
else:
    key_features = features

print(f"\nSelected {len(key_features)} key features for detailed analysis")
print("Key features:", key_features)

# Create a heatmap of cluster centers for key features
plt.figure(figsize=(14, 8))
key_cluster_centers = cluster_analysis[key_features]
# Normalize the data for better visualization
scaler = StandardScaler()
key_cluster_centers_scaled = pd.DataFrame(
    scaler.fit_transform(key_cluster_centers),
    index=key_cluster_centers.index,
    columns=key_cluster_centers.columns
)

# Create a heatmap
sns.heatmap(key_cluster_centers_scaled, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Normalized Cluster Centers for Key Features (Video Games)')
plt.ylabel('Cluster')
plt.tight_layout()
plt.savefig('clustering_results/kmeans_key_features_heatmap.png')
plt.close()

# Regional sales comparison by cluster if those columns exist
regional_cols = [col for col in features if any(region in col for region in ['na_', 'jp_', 'pal_', 'other_']) and 'sales' in col]
if regional_cols:
    print("\nAnalyzing regional sales patterns by cluster...")
    regional_means = df_cluster.groupby('Cluster')[regional_cols].mean()
    
    # Create a radar chart for regional sales comparison
    plt.figure(figsize=(12, 10))
    
    # Set plot parameters
    categories = regional_cols
    N = len(categories)
    
    # Create angle for each feature
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Close the loop
    
    # Initialize the plot
    ax = plt.subplot(111, polar=True)
    
    # Add each cluster
    for i in range(optimal_k):
        values = regional_means.iloc[i].values.tolist()
        values += values[:1]  # Close the loop
        
        # Plot values
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=f'Cluster {i}')
        ax.fill(angles, values, alpha=0.1)
    
    # Fix axis to go in the right order and start at 12 o'clock
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    
    # Draw axis lines for each angle and label
    plt.xticks(angles[:-1], categories)
    
    # Add legend
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title('Regional Sales Patterns by Cluster', size=15)
    plt.tight_layout()
    plt.savefig('clustering_results/kmeans_regional_sales_radar.png')
    plt.close()

# Create a parallel coordinates plot for cluster visualization
plt.figure(figsize=(15, 8))
# Get a subset of data for parallel coordinates plot (can be too dense with all data)
sample_size = min(1000, len(df_cluster))
sample_indices = np.random.choice(len(df_cluster), sample_size, replace=False)
sample_df = df_cluster.iloc[sample_indices].copy()

# Standardize the data for parallel coordinates plot
features_for_parallel = key_features[:7] if len(key_features) > 7 else key_features  # Limit to 7 features for readability
scaler = StandardScaler()
sample_df_scaled = sample_df.copy()
sample_df_scaled[features_for_parallel] = scaler.fit_transform(sample_df[features_for_parallel])

# Create parallel coordinates plot
pd.plotting.parallel_coordinates(
    sample_df_scaled, 'Cluster', 
    cols=features_for_parallel,
    color=plt.cm.viridis(np.linspace(0, 1, optimal_k))
)
plt.title('Parallel Coordinates Plot of Video Game Clusters')
plt.grid(False)
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('clustering_results/kmeans_parallel_coordinates.png')
plt.close()

# Summarize the characteristics of each cluster
print("\nCluster Characteristics Summary:")
for cluster in range(optimal_k):
    print(f"\nCluster {cluster}:")
    # Get the top 5 distinctive features for this cluster (highest absolute z-scores)
    cluster_features = key_cluster_centers_scaled.loc[cluster].abs().sort_values(ascending=False)
    top_features = cluster_features.head(5).index.tolist()
    
    for feature in top_features:
        raw_value = key_cluster_centers.loc[cluster, feature]
        scaled_value = key_cluster_centers_scaled.loc[cluster, feature]
        direction = "high" if scaled_value > 0 else "low"
        print(f"  - {feature}: {direction} ({raw_value:.2f}, z-score: {scaled_value:.2f})")
    
    # Optional: Display additional cluster characteristics based on available metrics
    if regional_cols:
        dominant_region = regional_means.loc[cluster].idxmax()
        print(f"  - Dominant sales region: {dominant_region}")
        
    # Add any other relevant information about the clusters
    cluster_size = cluster_counts[cluster]
    cluster_percentage = cluster_size / len(df_cluster) * 100
    print(f"  - Size: {cluster_size} games ({cluster_percentage:.2f}% of dataset)")

# Sample titles from each cluster
print("\nSample titles from each cluster:")
for cluster in range(optimal_k):
    if 'title' in df.columns:
        sample_titles = df_cluster[df_cluster['Cluster'] == cluster]['title'].sample(min(5, cluster_counts[cluster])).tolist()
        print(f"\nCluster {cluster} sample titles:")
        for title in sample_titles:
            print(f"  - {title}")

print("\nK-means clustering analysis complete. Results saved to 'clustering_results' directory.") 

Loading preprocessed video game sales dataset...
Dataset shape: (4000, 24)

Checking for non-numeric columns...
Found 16 numeric columns out of 24 total columns
Selected 16 features for clustering
Features: ['critic_score', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales', 'release_year', 'console_freq', 'genre_freq', 'publisher_freq', 'developer_freq', 'na_sales_ratio', 'jp_sales_ratio', 'pal_sales_ratio', 'game_age', 'sales_per_year']

Too many features, selecting most important ones...
Using 12 key features
Key features: ['critic_score', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales', 'release_year', 'na_sales_ratio', 'jp_sales_ratio', 'pal_sales_ratio', 'game_age', 'sales_per_year']

Finding optimal number of clusters using Elbow Method...
For n_clusters = 2, silhouette score is 0.893
For n_clusters = 3, silhouette score is 0.902
For n_clusters = 4, silhouette score is 0.882
For n_clusters = 5, silhouette score is 0.881
For n_clusters = 6, silhou

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import os

# Create directory for hierarchical clustering results
os.makedirs('hierarchical_results', exist_ok=True)

# Load the preprocessed data
print("Loading preprocessed video game sales dataset...")
df = pd.read_csv('processed_data/vgchartz_processed.csv')
print(f"Dataset shape: {df.shape}")

# Ensure we only have numeric data for clustering
print("\nChecking for non-numeric columns...")
numeric_df = df.select_dtypes(include=['float64', 'int64'])
numeric_columns = numeric_df.columns.tolist()
print(f"Found {len(numeric_columns)} numeric columns out of {len(df.columns)} total columns")

# Also exclude any identifier columns if present
features = [col for col in numeric_columns if 'id' not in col.lower()]
print(f"Selected {len(features)} features for clustering")
print("Features:", features)

# Use a subset of features if there are too many
if len(features) > 15:
    print("\nToo many features, selecting most important ones...")
    # Prioritize key metrics for clustering
    key_features = [col for col in features if any(term in col.lower() for term in 
                   ['sales', 'score', 'year', 'age', 'ratio'])]
    if len(key_features) >= 5:  # Ensure we have a reasonable number of features
        features = key_features
        print(f"Using {len(features)} key features")
        print("Key features:", features)

# Verify there are no NaN values in the selected features
X_df = df[features].copy()
nan_counts = X_df.isna().sum()
if nan_counts.sum() > 0:
    print("\nWarning: Found NaN values in features. Filling with feature means.")
    X_df = X_df.fillna(X_df.mean())

# Check for string values that might have been encoded as objects
for col in X_df.columns:
    if X_df[col].dtype == 'object':
        print(f"Converting column {col} to numeric, errors will be set to NaN")
        X_df[col] = pd.to_numeric(X_df[col], errors='coerce')
        X_df[col] = X_df[col].fillna(X_df[col].mean())

# Standardize the data for hierarchical clustering
print("\nStandardizing features...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_df)

# Sample data if too large (hierarchical clustering can be memory intensive)
max_samples = 1000  # Adjust based on your system's capabilities
if len(X_scaled) > max_samples:
    print(f"\nDataset too large for hierarchical clustering. Sampling {max_samples} records...")
    sample_indices = np.random.choice(len(X_scaled), max_samples, replace=False)
    X_scaled_sample = X_scaled[sample_indices]
    df_sample = df.iloc[sample_indices].copy()
    print(f"Working with sampled dataset of shape: {X_scaled_sample.shape}")
else:
    X_scaled_sample = X_scaled
    df_sample = df.copy()

# Compute the linkage matrix using Ward's method
print("Computing linkage matrix with Ward's method...")
Z = linkage(X_scaled_sample, method='ward')

# Plot the dendrogram to visualize hierarchical structure
plt.figure(figsize=(12, 8))
plt.title('Hierarchical Clustering Dendrogram for Video Game Sales')
plt.xlabel('Sample index')
plt.ylabel('Distance')
dendrogram(
    Z,
    truncate_mode='lastp',  # show only the last p merged clusters
    p=12,  # show only the last p merged clusters
    leaf_rotation=90.,
    leaf_font_size=8.,
    show_contracted=True,  # to get a distribution impression in truncated branches
)
plt.savefig('hierarchical_results/hierarchical_dendrogram.png')
plt.close()

# Evaluate different numbers of clusters using silhouette score
print("\nEvaluating different numbers of clusters...")
silhouette_scores = []
max_clusters = 10

for k in range(2, max_clusters + 1):
    # Get cluster labels
    labels = fcluster(Z, k, criterion='maxclust')
    
    # Calculate silhouette score
    silhouette_avg = silhouette_score(X_scaled_sample, labels)
    silhouette_scores.append(silhouette_avg)
    print(f"For n_clusters = {k}, silhouette score is {silhouette_avg:.3f}")

# Plot silhouette scores
plt.figure(figsize=(10, 6))
plt.plot(range(2, max_clusters + 1), silhouette_scores, marker='o')
plt.title('Silhouette Score Method for Hierarchical Clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.grid(True)
plt.savefig('hierarchical_results/hierarchical_silhouette_scores.png')
plt.close()

# Choose optimal number of clusters based on silhouette scores
optimal_k = silhouette_scores.index(max(silhouette_scores)) + 2  # +2 because we started from k=2
print(f"\nOptimal number of clusters based on silhouette score: {optimal_k}")

# Apply hierarchical clustering with the optimal number of clusters
print(f"\nPerforming hierarchical clustering with {optimal_k} clusters...")
labels = fcluster(Z, optimal_k, criterion='maxclust')
df_sample['Cluster'] = labels - 1  # Convert to 0-indexed clusters

# Count instances in each cluster
cluster_counts = df_sample['Cluster'].value_counts().sort_index()
print("\nCluster distribution:")
for cluster, count in cluster_counts.items():
    print(f"Cluster {cluster}: {count} instances ({count/len(df_sample)*100:.2f}%)")

# Analyze clusters
print("\nAnalyzing clusters...")
cluster_analysis = df_sample.groupby('Cluster')[features].mean()
print("\nCluster centers (mean values for each feature):")
print(cluster_analysis)

# Save cluster analysis to CSV
cluster_analysis.to_csv('hierarchical_results/hierarchical_cluster_analysis.csv')

# Visualize clusters using PCA for dimensionality reduction
print("\nVisualizing clusters using PCA...")
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled_sample)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df_sample['Cluster'], cmap='viridis', alpha=0.7)
plt.colorbar(scatter, label='Cluster')
plt.title(f'Video Game Sales Clusters Visualization with PCA (Hierarchical, k={optimal_k})')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.grid(True, alpha=0.3)
plt.savefig('hierarchical_results/hierarchical_pca_visualization.png')
plt.close()

# Analyze relationship between clusters and sales
print("\nAnalyzing relationship between clusters and sales...")
if 'total_sales' in features:
    sales_by_cluster = df_sample.groupby('Cluster')['total_sales'].mean().sort_values()
    print("\nAverage total sales by cluster:")
    print(sales_by_cluster)

    plt.figure(figsize=(10, 6))
    sales_by_cluster.plot(kind='bar')
    plt.title('Average Total Sales by Cluster')
    plt.xlabel('Cluster')
    plt.ylabel('Average Sales')
    plt.axhline(y=df_sample['total_sales'].mean(), color='red', linestyle='--', label='Overall Average')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('hierarchical_results/hierarchical_sales_by_cluster.png')
    plt.close()

# Analyze relationship between clusters and critic scores if available
if 'critic_score' in features:
    scores_by_cluster = df_sample.groupby('Cluster')['critic_score'].mean().sort_values()
    print("\nAverage critic scores by cluster:")
    print(scores_by_cluster)

    plt.figure(figsize=(10, 6))
    scores_by_cluster.plot(kind='bar')
    plt.title('Average Critic Score by Cluster')
    plt.xlabel('Cluster')
    plt.ylabel('Average Critic Score')
    plt.axhline(y=df_sample['critic_score'].mean(), color='red', linestyle='--', label='Overall Average')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('hierarchical_results/hierarchical_scores_by_cluster.png')
    plt.close()

# Analyze key features by cluster
# Select a subset of important features to visualize
if len(features) > 10:
    # Get the most variable features across clusters
    feature_variance = cluster_analysis.var().sort_values(ascending=False)
    key_features = feature_variance.head(10).index.tolist()
else:
    key_features = features

print(f"\nSelected {len(key_features)} key features for detailed analysis")
print("Key features:", key_features)

# Create a heatmap of cluster centers for key features
plt.figure(figsize=(14, 8))
key_cluster_centers = cluster_analysis[key_features]
# Normalize the data for better visualization
key_cluster_centers_scaled = pd.DataFrame(
    scaler.fit_transform(key_cluster_centers),
    index=key_cluster_centers.index,
    columns=key_cluster_centers.columns
)

# Create a heatmap
sns.heatmap(key_cluster_centers_scaled, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Normalized Cluster Centers for Key Features (Video Games)')
plt.ylabel('Cluster')
plt.tight_layout()
plt.savefig('hierarchical_results/hierarchical_key_features_heatmap.png')
plt.close()

# Regional sales comparison by cluster if those columns exist
regional_cols = [col for col in features if any(region in col for region in ['na_', 'jp_', 'pal_', 'other_']) and 'sales' in col]
if regional_cols:
    print("\nAnalyzing regional sales patterns by cluster...")
    regional_means = df_sample.groupby('Cluster')[regional_cols].mean()
    
    # Create a radar chart for regional sales comparison
    plt.figure(figsize=(12, 10))
    
    # Set plot parameters
    categories = regional_cols
    N = len(categories)
    
    # Create angle for each feature
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Close the loop
    
    # Initialize the plot
    ax = plt.subplot(111, polar=True)
    
    # Add each cluster
    for i in range(optimal_k):
        if i in regional_means.index:  # Check if the cluster exists
            values = regional_means.loc[i].values.tolist()
            values += values[:1]  # Close the loop
            
            # Plot values
            ax.plot(angles, values, linewidth=2, linestyle='solid', label=f'Cluster {i}')
            ax.fill(angles, values, alpha=0.1)
    
    # Fix axis to go in the right order and start at 12 o'clock
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    
    # Draw axis lines for each angle and label
    plt.xticks(angles[:-1], categories)
    
    # Add legend
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title('Regional Sales Patterns by Cluster', size=15)
    plt.tight_layout()
    plt.savefig('hierarchical_results/hierarchical_regional_sales_radar.png')
    plt.close()

# Create a parallel coordinates plot for cluster visualization
plt.figure(figsize=(15, 8))
# Get a subset of data for parallel coordinates plot (can be too dense with all data)
sample_size = min(500, len(df_sample))
if len(df_sample) > sample_size:
    sample_indices = np.random.choice(len(df_sample), sample_size, replace=False)
    sample_df = df_sample.iloc[sample_indices].copy()
else:
    sample_df = df_sample.copy()

# Standardize the data for parallel coordinates plot
features_for_parallel = key_features[:7] if len(key_features) > 7 else key_features  # Limit to 7 features for readability
sample_df_scaled = sample_df.copy()
sample_df_scaled[features_for_parallel] = scaler.fit_transform(sample_df[features_for_parallel])

# Create parallel coordinates plot
pd.plotting.parallel_coordinates(
    sample_df_scaled, 'Cluster', 
    cols=features_for_parallel,
    colormap='viridis'
)
plt.title('Parallel Coordinates Plot of Video Game Clusters')
plt.grid(False)
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('hierarchical_results/hierarchical_parallel_coordinates.png')
plt.close()

# Summarize the characteristics of each cluster
print("\nCluster Characteristics Summary:")
for cluster in range(optimal_k):
    if cluster not in cluster_analysis.index:
        continue  # Skip clusters that don't exist (possible if using 0-indexed)
    
    print(f"\nCluster {cluster}:")
    # Get the top 5 distinctive features for this cluster (highest absolute z-scores)
    if cluster in key_cluster_centers_scaled.index:
        cluster_features = key_cluster_centers_scaled.loc[cluster].abs().sort_values(ascending=False)
        top_features = cluster_features.head(5).index.tolist()
        
        for feature in top_features:
            raw_value = key_cluster_centers.loc[cluster, feature]
            scaled_value = key_cluster_centers_scaled.loc[cluster, feature]
            direction = "high" if float(scaled_value) > 0 else "low"
            print(f"  - {feature}: {direction} ({raw_value:.2f}, z-score: {scaled_value:.2f})")
    
    # Optional: Display additional cluster characteristics based on available metrics
    if regional_cols and cluster in regional_means.index:
        dominant_region = regional_means.loc[cluster].idxmax()
        print(f"  - Dominant sales region: {dominant_region}")
        
    # Add any other relevant information about the clusters
    if cluster in cluster_counts.index:
        cluster_size = cluster_counts[cluster]
        cluster_percentage = cluster_size / len(df_sample) * 100
        print(f"  - Size: {cluster_size} games ({cluster_percentage:.2f}% of dataset)")

# Sample titles from each cluster if title column exists
if 'title' in df.columns:
    print("\nSample titles from each cluster:")
    for cluster in range(optimal_k):
        if cluster in df_sample['Cluster'].values:  # Check if cluster exists in results
            cluster_games = df_sample[df_sample['Cluster'] == cluster]
            sample_size = min(5, len(cluster_games))
            if sample_size > 0:
                sample_titles = cluster_games['title'].sample(sample_size).tolist()
                print(f"\nCluster {cluster} sample titles:")
                for title in sample_titles:
                    print(f"  - {title}")

# Create correlation matrix between original features and PCA components
print("\nCreating correlation matrix between features and PCA components...")
pca_components = pd.DataFrame(
    pca.components_.T, 
    columns=[f'PC{i+1}' for i in range(2)],
    index=features
)
plt.figure(figsize=(10, 12))
sns.heatmap(pca_components, cmap='coolwarm', annot=True, fmt='.2f')
plt.title('Feature Correlation with Principal Components')
plt.tight_layout()
plt.savefig('hierarchical_results/hierarchical_pca_correlation.png')
plt.close()

print("\nHierarchical clustering analysis complete. Results saved to 'hierarchical_results' directory.")

Loading preprocessed video game sales dataset...
Dataset shape: (4000, 24)

Checking for non-numeric columns...
Found 16 numeric columns out of 24 total columns
Selected 16 features for clustering
Features: ['critic_score', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales', 'release_year', 'console_freq', 'genre_freq', 'publisher_freq', 'developer_freq', 'na_sales_ratio', 'jp_sales_ratio', 'pal_sales_ratio', 'game_age', 'sales_per_year']

Too many features, selecting most important ones...
Using 12 key features
Key features: ['critic_score', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales', 'release_year', 'na_sales_ratio', 'jp_sales_ratio', 'pal_sales_ratio', 'game_age', 'sales_per_year']

Standardizing features...

Dataset too large for hierarchical clustering. Sampling 1000 records...
Working with sampled dataset of shape: (1000, 12)
Computing linkage matrix with Ward's method...

Evaluating different numbers of clusters...
For n_clusters = 2, silhou

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import os
from datetime import datetime

# Create directory for decision tree results
os.makedirs('decision_tree_results', exist_ok=True)

# Load the preprocessed data
print("Loading preprocessed video game sales dataset...")
df = pd.read_csv('processed_data/vgchartz_processed.csv')
print(f"Dataset shape: {df.shape}")

# Handle date columns if they exist
date_columns = ['release_date', 'last_update']
for col in date_columns:
    if col in df.columns:
        print(f"Converting {col} to datetime and extracting year")
        try:
            # Try to convert to datetime and extract year as a numeric feature
            df[col] = pd.to_datetime(df[col], errors='coerce')
            df[f'{col}_year'] = df[col].dt.year
            df[f'{col}_month'] = df[col].dt.month
            
            # Drop original date column as it cannot be used directly
            df.drop(col, axis=1, inplace=True)
        except:
            print(f"Error processing {col}, dropping it")
            df.drop(col, axis=1, inplace=True)

# Need to create a target variable since this is sales data
# Let's create a categorical target based on total sales
# Create high/low sales binary classification (1 for high sales, 0 for low sales)
# Using median as threshold to ensure balanced classes
sales_median = df['total_sales'].median()
df['high_sales'] = (df['total_sales'] > sales_median).astype(int)
print(f"High sales threshold (median): {sales_median:.2f}")
print(f"High sales prevalence: {df['high_sales'].mean():.2%}")

# Select features for classification
# Remove identifier columns and target
exclude_cols = ['title', 'platform', 'genre', 'publisher', 'developer', 'high_sales', 'total_sales']

# Also remove any features that are directly derived from total_sales to avoid data leakage
derived_cols = ['na_sales', 'jp_sales', 'pal_sales', 'other_sales', 'sales_per_year']
exclude_cols.extend(derived_cols)

print(f"Excluding columns: {exclude_cols}")
features = [col for col in df.columns if col not in exclude_cols]

# Check for NaN values in the features and fill them
nan_counts = df[features].isna().sum()
if nan_counts.sum() > 0:
    print("\nFound NaN values in features. Filling with medians.")
    for col in features:
        if nan_counts[col] > 0:
            df[col] = df[col].fillna(df[col].median())

# Check for non-numeric columns
non_numeric_cols = df[features].select_dtypes(exclude=['number']).columns.tolist()
if non_numeric_cols:
    print(f"\nFound non-numeric columns: {non_numeric_cols}")
    print("Dropping non-numeric columns for classification")
    features = [col for col in features if col not in non_numeric_cols]

X = df[features].values
y = df['high_sales'].values

print(f"Selected {len(features)} features for classification")
print("Features:", features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a decision tree with default parameters
print("\nTraining a simple decision tree classifier...")
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train_scaled, y_train)

# Evaluate on test set
y_pred = dt_clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Decision tree accuracy: {accuracy:.4f}")

# Print classification report
print("\nClassification Report:")
report = classification_report(y_test, y_pred)
print(report)

# Save classification report to file
with open('decision_tree_results/classification_report.txt', 'w') as f:
    f.write("Classification Report for Decision Tree on Video Game Sales:\n")
    f.write(str(report))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Low Sales', 'High Sales'],
            yticklabels=['Low Sales', 'High Sales'])
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix for Decision Tree')
plt.tight_layout()
plt.savefig('decision_tree_results/confusion_matrix.png')
plt.close()

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': dt_clf.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 10 Important Features:")
print(feature_importance.head(min(10, len(features))))

# Save feature importance to CSV
feature_importance.to_csv('decision_tree_results/feature_importance.csv', index=False)

# Plot feature importance (top 15)
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(min(15, len(features)))
sns.barplot(x='Importance', y='Feature', data=top_features)
plt.title('Top Features by Importance')
plt.tight_layout()
plt.savefig('decision_tree_results/feature_importance.png')
plt.close()

# Hyperparameter tuning using GridSearchCV
print("\nPerforming hyperparameter tuning for decision tree...")
param_grid = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

grid_search = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1
)

grid_search.fit(X_train_scaled, y_train)

# Print best parameters
print("Best parameters:", grid_search.best_params_)
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

# Train model with best parameters
best_dt = grid_search.best_estimator_
best_dt.fit(X_train_scaled, y_train)

# Evaluate optimized model
y_pred_best = best_dt.predict(X_test_scaled)
best_accuracy = accuracy_score(y_test, y_pred_best)
print(f"Optimized decision tree accuracy: {best_accuracy:.4f}")

# Print classification report for optimized model
print("\nClassification Report (Optimized Model):")
best_report = classification_report(y_test, y_pred_best)
print(best_report)

# Save classification report for optimized model
with open('decision_tree_results/optimized_classification_report.txt', 'w') as f:
    f.write("Classification Report for Optimized Decision Tree on Video Game Sales:\n")
    f.write(str(best_report))

# Confusion matrix for optimized model
best_conf_matrix = confusion_matrix(y_test, y_pred_best)
plt.figure(figsize=(8, 6))
sns.heatmap(best_conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Low Sales', 'High Sales'],
            yticklabels=['Low Sales', 'High Sales'])
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix for Optimized Decision Tree')
plt.tight_layout()
plt.savefig('decision_tree_results/optimized_confusion_matrix.png')
plt.close()

# Visualize the optimized decision tree (if not too large)
max_depth_for_visualization = 3  # Limit for visualization

# If tree is too deep, create a simpler one just for visualization
vis_dt = DecisionTreeClassifier(max_depth=max_depth_for_visualization, random_state=42)
vis_dt.fit(X_train_scaled, y_train)
tree_to_visualize = vis_dt
tree_title = f"Decision Tree (Limited to depth {max_depth_for_visualization} for visualization)"

# Plot the tree
plt.figure(figsize=(20, 15))
plot_tree(
    tree_to_visualize,
    feature_names=features,
    class_names=['Low Sales', 'High Sales'],
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title(tree_title)
plt.savefig('decision_tree_results/decision_tree_visualization.png', dpi=300, bbox_inches='tight')
plt.close()

# Export text representation of the tree
tree_text = export_text(
    tree_to_visualize,
    feature_names=features
)

with open('decision_tree_results/decision_tree_text.txt', 'w') as f:
    f.write(tree_text)

# Check if we have any sales ratio columns left after removing derived columns
regional_ratio_cols = [col for col in features if '_sales_ratio' in col]
if regional_ratio_cols:
    # Create regional sales ratio plot by sales class
    plt.figure(figsize=(10, 6))
    sales_class_means = df.groupby('high_sales')[regional_ratio_cols].mean()
    
    sales_class_means.T.plot(kind='bar')
    plt.title('Regional Sales Ratio by Sales Class')
    plt.xlabel('Region')
    plt.ylabel('Average Sales Ratio')
    plt.legend(['Low Sales', 'High Sales'])
    plt.tight_layout()
    plt.savefig('decision_tree_results/regional_sales_by_class.png')
    plt.close()

# Create feature distributions by sales class
top_5_features = feature_importance.head(min(5, len(features)))['Feature'].tolist()
for feature in top_5_features:
    plt.figure(figsize=(10, 6))
    # Using histplot without KDE to avoid errors
    sns.histplot(data=df, x=feature, hue='high_sales', bins=20, element='step', kde=False)
    plt.title(f'Distribution of {feature} by Sales Class')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig(f'decision_tree_results/feature_distribution_{feature}.png')
    plt.close()

# Create correlation matrix for features
corr_matrix = df[features + ['high_sales']].corr()
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, cmap='coolwarm', annot=True, fmt='.2f', 
            square=True, linewidths=.5)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.savefig('decision_tree_results/correlation_matrix.png')
plt.close()

print("\nDecision tree analysis complete. Results saved to 'decision_tree_results' directory.")

# Create a function for predicting new games' sales class
def predict_sales_class(sample_data, model=best_dt, scaler=scaler, features=features):
    """
    Predict sales class for a new video game.
    
    Parameters:
    sample_data : dict
        Dictionary with feature names and values
    model : trained model
        Trained decision tree model
    scaler : fitted scaler
        Fitted StandardScaler
    features : list
        List of feature names
    
    Returns:
    prediction : int
        0 for low sales, 1 for high sales
    prob : float
        Probability of high sales
    """
    # Convert sample to array in the correct order
    sample_array = np.array([sample_data.get(feature, 0) for feature in features]).reshape(1, -1)
    
    # Scale the sample
    sample_scaled = scaler.transform(sample_array)
    
    # Predict
    prediction = model.predict(sample_scaled)[0]
    probability = model.predict_proba(sample_scaled)[0][1]
    
    return prediction, probability

# Example usage
print("\nExample prediction:")
# Create a sample (you should replace this with actual feature values)
sample = {
    'critic_score': 80,
    'release_year': 2018,
    'console_freq': 5,
    'genre_freq': 10,
    'publisher_freq': 3,
    # Add other features as needed
}

# Print only features in the sample that match our feature list
present_features = {k: v for k, v in sample.items() if k in features}
print(f"Sample features: {present_features}")

# Make a prediction
try:
    pred, prob = predict_sales_class(sample)
    print(f"Prediction: {'High Sales' if pred == 1 else 'Low Sales'}")
    print(f"Probability of high sales: {prob:.2%}")
except Exception as e:
    print(f"Error making prediction: {e}")
    print("This is just an example. You may need to provide values for all required features.") 

Loading preprocessed video game sales dataset...
Dataset shape: (4000, 24)
Converting release_date to datetime and extracting year
Converting last_update to datetime and extracting year
High sales threshold (median): -0.54
High sales prevalence: 48.25%
Excluding columns: ['title', 'platform', 'genre', 'publisher', 'developer', 'high_sales', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales', 'sales_per_year']

Found non-numeric columns: ['rating_Poor', 'rating_Average', 'rating_Good', 'rating_Great', 'rating_Excellent']
Dropping non-numeric columns for classification
Selected 14 features for classification
Features: ['critic_score', 'release_year', 'console_freq', 'genre_freq', 'publisher_freq', 'developer_freq', 'na_sales_ratio', 'jp_sales_ratio', 'pal_sales_ratio', 'game_age', 'release_date_year', 'release_date_month', 'last_update_year', 'last_update_month']
Training set: 3000 samples
Test set: 1000 samples

Training a simple decision tree classifier...
Decision tree 

<Figure size 1000x600 with 0 Axes>

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
import os

# Create directory for Naive Bayes results
os.makedirs('naive_bayes_results', exist_ok=True)

# Load the preprocessed data
print("Loading preprocessed video game sales dataset...")
df = pd.read_csv('processed_data/vgchartz_processed.csv')
print(f"Dataset shape: {df.shape}")

# Handle date columns if they exist
date_columns = ['release_date', 'last_update']
for col in date_columns:
    if col in df.columns:
        print(f"Converting {col} to datetime and extracting year")
        try:
            # Try to convert to datetime and extract year as a numeric feature
            df[col] = pd.to_datetime(df[col], errors='coerce')
            df[f'{col}_year'] = df[col].dt.year
            df[f'{col}_month'] = df[col].dt.month
            
            # Drop original date column as it cannot be used directly
            df.drop(col, axis=1, inplace=True)
        except:
            print(f"Error processing {col}, dropping it")
            df.drop(col, axis=1, inplace=True)

# Create high/low sales binary classification (1 for high sales, 0 for low sales)
# Using median as threshold to ensure balanced classes
sales_median = df['total_sales'].median()
df['high_sales'] = (df['total_sales'] > sales_median).astype(int)
print(f"High sales threshold (median): {sales_median:.2f}")
print(f"High sales prevalence: {df['high_sales'].mean():.2%}")

# Select features for classification
# Remove identifier columns and target
exclude_cols = ['title', 'platform', 'genre', 'publisher', 'developer', 'high_sales', 'total_sales']

# Also remove any features that are directly derived from total_sales to avoid data leakage
derived_cols = ['na_sales', 'jp_sales', 'pal_sales', 'other_sales', 'sales_per_year']
exclude_cols.extend(derived_cols)

print(f"Excluding columns: {exclude_cols}")
features = [col for col in df.columns if col not in exclude_cols]

# Check for NaN values in the features and fill them
nan_counts = df[features].isna().sum()
if nan_counts.sum() > 0:
    print("\nFound NaN values in features. Filling with medians.")
    for col in features:
        if nan_counts[col] > 0:
            df[col] = df[col].fillna(df[col].median())

# Check for non-numeric columns
non_numeric_cols = df[features].select_dtypes(exclude=['number']).columns.tolist()
if non_numeric_cols:
    print(f"\nFound non-numeric columns: {non_numeric_cols}")
    print("Dropping non-numeric columns for classification")
    features = [col for col in features if col not in non_numeric_cols]

X = df[features].values
y = df['high_sales'].values

print(f"Selected {len(features)} features for classification")
print("Features:", features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Scale features - MinMaxScaler tends to work better with Naive Bayes than StandardScaler
# as it preserves the distribution shape while bounding values
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -------------------------
# Gaussian Naive Bayes
# -------------------------
print("\n---- Gaussian Naive Bayes ----")
gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train)

# Evaluate on test set
y_pred_gnb = gnb.predict(X_test_scaled)
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
print(f"Gaussian Naive Bayes accuracy: {accuracy_gnb:.4f}")

# Probability predictions for ROC curve
y_pred_prob_gnb = gnb.predict_proba(X_test_scaled)[:, 1]

# Print classification report
print("\nClassification Report (Gaussian Naive Bayes):")
gnb_report = classification_report(y_test, y_pred_gnb)
print(gnb_report)

# Save classification report to file
with open('naive_bayes_results/gnb_classification_report.txt', 'w') as f:
    f.write("Classification Report for Gaussian Naive Bayes on Video Game Sales:\n")
    f.write(str(gnb_report))

# Confusion matrix
conf_matrix_gnb = confusion_matrix(y_test, y_pred_gnb)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_gnb, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Low Sales', 'High Sales'],
            yticklabels=['Low Sales', 'High Sales'])
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix for Gaussian Naive Bayes')
plt.tight_layout()
plt.savefig('naive_bayes_results/gnb_confusion_matrix.png')
plt.close()

# -------------------------
# Bernoulli Naive Bayes 
# -------------------------
print("\n---- Bernoulli Naive Bayes ----")
# Bernoulli NB works with binary features, so we need to consider this fact
bnb = BernoulliNB()
bnb.fit(X_train_scaled, y_train)

# Evaluate on test set
y_pred_bnb = bnb.predict(X_test_scaled)
accuracy_bnb = accuracy_score(y_test, y_pred_bnb)
print(f"Bernoulli Naive Bayes accuracy: {accuracy_bnb:.4f}")

# Probability predictions for ROC curve
y_pred_prob_bnb = bnb.predict_proba(X_test_scaled)[:, 1]

# Print classification report
print("\nClassification Report (Bernoulli Naive Bayes):")
bnb_report = classification_report(y_test, y_pred_bnb)
print(bnb_report)

# Save classification report to file
with open('naive_bayes_results/bnb_classification_report.txt', 'w') as f:
    f.write("Classification Report for Bernoulli Naive Bayes on Video Game Sales:\n")
    f.write(str(bnb_report))

# Confusion matrix
conf_matrix_bnb = confusion_matrix(y_test, y_pred_bnb)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_bnb, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Low Sales', 'High Sales'],
            yticklabels=['Low Sales', 'High Sales'])
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix for Bernoulli Naive Bayes')
plt.tight_layout()
plt.savefig('naive_bayes_results/bnb_confusion_matrix.png')
plt.close()

# -------------------------
# Cross-validation for both models
# -------------------------
print("\n---- Cross-validation ----")

# Cross-validation for Gaussian NB
cv_scores_gnb = cross_val_score(gnb, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Gaussian NB 5-fold CV accuracy: {cv_scores_gnb.mean():.4f} ± {cv_scores_gnb.std():.4f}")

# Cross-validation for Bernoulli NB
cv_scores_bnb = cross_val_score(bnb, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Bernoulli NB 5-fold CV accuracy: {cv_scores_bnb.mean():.4f} ± {cv_scores_bnb.std():.4f}")

# -------------------------
# ROC Curves and AUC
# -------------------------
plt.figure(figsize=(10, 8))

# ROC for Gaussian NB
fpr_gnb, tpr_gnb, _ = roc_curve(y_test, y_pred_prob_gnb)
roc_auc_gnb = auc(fpr_gnb, tpr_gnb)
plt.plot(fpr_gnb, tpr_gnb, label=f'Gaussian NB (AUC = {roc_auc_gnb:.3f})')

# ROC for Bernoulli NB
fpr_bnb, tpr_bnb, _ = roc_curve(y_test, y_pred_prob_bnb)
roc_auc_bnb = auc(fpr_bnb, tpr_bnb)
plt.plot(fpr_bnb, tpr_bnb, label=f'Bernoulli NB (AUC = {roc_auc_bnb:.3f})')

# Reference line (random classifier)
plt.plot([0, 1], [0, 1], 'k--')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Naive Bayes Classifiers')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.savefig('naive_bayes_results/roc_curves.png')
plt.close()

# -------------------------
# Feature importance for Naive Bayes
# -------------------------
# For Naive Bayes, we can look at the difference in log probability between classes
# This gives us an indication of how much each feature contributes to the classification

# Function to calculate feature importance for Gaussian NB
def compute_feature_importance_gnb(model, feature_names):
    # Get the feature means for each class
    theta_0 = model.theta_[0]  # Mean for class 0
    theta_1 = model.theta_[1]  # Mean for class 1
    
    # Get the feature variances for each class
    sigma_0 = model.var_[0]  # Variance for class 0
    sigma_1 = model.var_[1]  # Variance for class 1
    
    # Calculate the absolute difference in means, normalized by variance
    # This gives a measure of how discriminative each feature is
    importance = np.abs(theta_1 - theta_0) / np.sqrt((sigma_0 + sigma_1) / 2)
    
    # Create DataFrame with feature names and importance scores
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance
    }).sort_values('Importance', ascending=False)
    
    return feature_importance

# Calculate feature importance for Gaussian NB
gnb_feature_importance = compute_feature_importance_gnb(gnb, features)

print("\nTop 10 Important Features (Gaussian NB):")
print(gnb_feature_importance.head(min(10, len(features))))

# Save feature importance to CSV
gnb_feature_importance.to_csv('naive_bayes_results/gnb_feature_importance.csv', index=False)

# Plot feature importance (top 15)
plt.figure(figsize=(12, 8))
top_features_gnb = gnb_feature_importance.head(min(15, len(features)))
sns.barplot(x='Importance', y='Feature', data=top_features_gnb)
plt.title('Top 15 Features by Importance (Gaussian NB)')
plt.tight_layout()
plt.savefig('naive_bayes_results/gnb_feature_importance.png')
plt.close()

# -------------------------
# Compare with decision tree results if available
# -------------------------
print("\n---- Comparison with Decision Tree ----")

# Create comparison table with the Naive Bayes models
comparison_data = {
    'Model': ['Gaussian NB', 'Bernoulli NB'],
    'Accuracy': [accuracy_gnb, accuracy_bnb],
    'CV Accuracy': [cv_scores_gnb.mean(), cv_scores_bnb.mean()],
    'AUC': [roc_auc_gnb, roc_auc_bnb]
}

# Try to load Decision Tree results from file
try:
    with open('decision_tree_results/optimized_classification_report.txt', 'r') as f:
        dt_report = f.read()
    print("Decision Tree results loaded from file.")
    
    # Add Decision Tree to comparison if we can extract accuracy
    # This is a simplistic approach; in a real application you might parse the report more carefully
    import re
    accuracy_match = re.search(r'accuracy\s*:\s*([\d\.]+)', dt_report)
    if accuracy_match:
        dt_accuracy = float(accuracy_match.group(1))
        comparison_data['Model'].insert(0, 'Decision Tree')
        comparison_data['Accuracy'].insert(0, dt_accuracy)
        comparison_data['CV Accuracy'].insert(0, None)
        comparison_data['AUC'].insert(0, None)
except FileNotFoundError:
    print("Decision Tree results file not found. Comparison will only include Naive Bayes models.")

# Create and display comparison DataFrame
comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.fillna('Not available')
print("\nModel Comparison:")
print(comparison_df)

# Save comparison to file
comparison_df.to_csv('naive_bayes_results/model_comparison.csv', index=False)

# -------------------------
# Create prediction function
# -------------------------
# Determine the best Naive Bayes model
best_nb_model = gnb if accuracy_gnb >= accuracy_bnb else bnb
best_nb_name = "Gaussian NB" if accuracy_gnb >= accuracy_bnb else "Bernoulli NB"
best_accuracy = accuracy_gnb if accuracy_gnb >= accuracy_bnb else accuracy_bnb
print(f"\nBest Naive Bayes model: {best_nb_name} (Accuracy: {best_accuracy:.4f})")

def predict_sales_class_nb(sample_data, model=best_nb_model, scaler=scaler, features=features):
    """
    Predict sales class for a new video game using Naive Bayes.
    
    Parameters:
    sample_data : dict
        Dictionary with feature names and values
    model : trained model
        Trained Naive Bayes model
    scaler : fitted scaler
        Fitted scaler
    features : list
        List of feature names
    
    Returns:
    prediction : int
        0 for low sales, 1 for high sales
    prob : float
        Probability of high sales
    """
    # Convert sample to array in the correct order
    sample_array = np.array([sample_data.get(feature, 0) for feature in features]).reshape(1, -1)
    
    # Scale the sample
    sample_scaled = scaler.transform(sample_array)
    
    # Predict
    prediction = model.predict(sample_scaled)[0]
    probability = model.predict_proba(sample_scaled)[0][1]
    
    return prediction, probability

# Example usage
print("\nExample prediction:")
# Create a sample (you should replace this with actual feature values)
sample = {
    'critic_score': 80,
    'release_date_year': 2018,
    'console_freq': 5,
    'genre_freq': 10,
    'publisher_freq': 3,
    # Add other features as needed
}

# Print only features in the sample that are in our feature list
present_features = {k: v for k, v in sample.items() if k in features}
print(f"Sample features: {present_features}")

# Make a prediction
try:
    pred, prob = predict_sales_class_nb(sample)
    print(f"Prediction: {'High Sales' if pred == 1 else 'Low Sales'}")
    print(f"Probability of high sales: {prob:.2%}")
except Exception as e:
    print(f"Error making prediction: {e}")
    print("This is just an example. You may need to provide values for all required features.")

# Additional analysis: Feature distributions by sales class
print("\nGenerating feature distribution plots...")
top_5_features = gnb_feature_importance.head(min(5, len(features)))['Feature'].tolist()
for feature in top_5_features:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x=feature, hue='high_sales', bins=20, element='step', kde=False)
    plt.title(f'Distribution of {feature} by Sales Class')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig(f'naive_bayes_results/feature_distribution_{feature}.png')
    plt.close()

print("\nNaive Bayes analysis complete. Results saved to 'naive_bayes_results' directory.") 

Loading preprocessed video game sales dataset...
Dataset shape: (4000, 24)
Converting release_date to datetime and extracting year
Converting last_update to datetime and extracting year
High sales threshold (median): -0.54
High sales prevalence: 48.25%
Excluding columns: ['title', 'platform', 'genre', 'publisher', 'developer', 'high_sales', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales', 'sales_per_year']

Found non-numeric columns: ['rating_Poor', 'rating_Average', 'rating_Good', 'rating_Great', 'rating_Excellent']
Dropping non-numeric columns for classification
Selected 14 features for classification
Features: ['critic_score', 'release_year', 'console_freq', 'genre_freq', 'publisher_freq', 'developer_freq', 'na_sales_ratio', 'jp_sales_ratio', 'pal_sales_ratio', 'game_age', 'release_date_year', 'release_date_month', 'last_update_year', 'last_update_month']
Training set: 3000 samples
Test set: 1000 samples

---- Gaussian Naive Bayes ----
Gaussian Naive Bayes accuracy